In [1]:
import os 
from dotenv import load_dotenv
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [4]:
gemma_model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
gemma_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018E5FA9E8C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018E5FA9C430>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
gemma_model.invoke([HumanMessage(content="Hi, My name is Andrew Adel and I am a Chief AI Engineer.")])

AIMessage(content="Hello Andrew Adel! It's nice to meet you. As a Chief AI Engineer, I imagine you're doing some fascinating work. \n\nWhat kind of projects are you currently involved in?  \n\nI'm always eager to learn more about the applications of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 24, 'total_tokens': 83, 'completion_time': 0.107272727, 'prompt_time': 0.001783458, 'queue_time': 0.186817591, 'total_time': 0.109056185}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2256105e-6466-46af-9cde-35bc651fe4fc-0', usage_metadata={'input_tokens': 24, 'output_tokens': 59, 'total_tokens': 83})

In [6]:
gemma_model.invoke(
    [
        HumanMessage(content="Hi, My name is Andrew Adel and I am a Chief AI Engineer."),
        AIMessage(content="It's nice to meet you, Andrew Adel! That's a fascinating title.\n\nAs a Chief AI Engineer, what are some of the most exciting projects you're working on right now?  \n\nI'm always eager to learn more about the innovative work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and What do I do?")
    ]
)

AIMessage(content="You told me your name is Andrew Adel and that you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm interested in learning more! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 110, 'total_tokens': 160, 'completion_time': 0.090909091, 'prompt_time': 0.003285848, 'queue_time': 0.187710637, 'total_time': 0.094194939}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d55d6e3-d4bd-448d-97b0-7f50a323dd88-0', usage_metadata={'input_tokens': 110, 'output_tokens': 50, 'total_tokens': 160})

In [7]:
store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [8]:
with_message_history = RunnableWithMessageHistory(runnable=gemma_model, get_session_history=get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000018E5FB91000>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [9]:
first_config = {"configurable": {"session_id": "chat1"}}
first_config

{'configurable': {'session_id': 'chat1'}}

In [10]:
first_gemma_response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Andrew Adel and I am a Chief AI Engineer.")],
    config=first_config
)
first_gemma_response

AIMessage(content="Hello Andrew Adel! It's nice to meet you.\n\nAs a Chief AI Engineer, I imagine you're working on some fascinating projects. What are you currently focusing on in the field of AI?  \n\nI'm always eager to learn more about the latest advancements and applications of artificial intelligence.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 24, 'total_tokens': 89, 'completion_time': 0.118181818, 'prompt_time': 0.001325279, 'queue_time': 0.188009474, 'total_time': 0.119507097}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--645ffb16-65db-4178-be16-1782c4f86cb3-0', usage_metadata={'input_tokens': 24, 'output_tokens': 65, 'total_tokens': 89})

In [11]:
first_gemma_response.content

"Hello Andrew Adel! It's nice to meet you.\n\nAs a Chief AI Engineer, I imagine you're working on some fascinating projects. What are you currently focusing on in the field of AI?  \n\nI'm always eager to learn more about the latest advancements and applications of artificial intelligence.\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=first_config
)

AIMessage(content='You told me your name is Andrew Adel.  \n\nIs there something else I can help you with? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 103, 'total_tokens': 129, 'completion_time': 0.047272727, 'prompt_time': 0.003515457, 'queue_time': 0.675239678, 'total_time': 0.050788184}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--41c754e4-a807-4d01-a0a8-688033f64ec8-0', usage_metadata={'input_tokens': 103, 'output_tokens': 26, 'total_tokens': 129})

In [13]:
second_config = {"configurable": {"session_id": "chat2"}}
second_config

{'configurable': {'session_id': 'chat2'}}

In [14]:
second_gemma_response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=second_config
)
second_gemma_response

AIMessage(content='As an AI, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 15, 'total_tokens': 48, 'completion_time': 0.06, 'prompt_time': 0.001253889, 'queue_time': 0.187116372, 'total_time': 0.061253889}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2dc20eb6-bc17-484d-b71f-bdd264406657-0', usage_metadata={'input_tokens': 15, 'output_tokens': 33, 'total_tokens': 48})

In [15]:
second_gemma_response.content

'As an AI, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n'

In [16]:
second_gemma_response = with_message_history.invoke(
    [HumanMessage(content="Hey my name is Tareq.")],
    config=second_config
)
second_gemma_response

AIMessage(content="Hello Tareq, it's nice to meet you!\n\nHow can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 63, 'total_tokens': 86, 'completion_time': 0.041818182, 'prompt_time': 0.002208478, 'queue_time': 0.187542364, 'total_time': 0.04402666}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--29c4d4e9-5992-48a8-b397-c1905f436b3b-0', usage_metadata={'input_tokens': 63, 'output_tokens': 23, 'total_tokens': 86})

In [17]:
second_gemma_response.content

"Hello Tareq, it's nice to meet you!\n\nHow can I help you today?\n"

In [18]:
second_gemma_response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=second_config
)
second_gemma_response

AIMessage(content='Your name is Tareq. 😊  \n\nIs there anything else I can help you with, Tareq? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 100, 'total_tokens': 126, 'completion_time': 0.047272727, 'prompt_time': 0.002656108, 'queue_time': 0.187534639, 'total_time': 0.049928835}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--34b334a2-af9e-4f6c-a3aa-ab62f78bf5f4-0', usage_metadata={'input_tokens': 100, 'output_tokens': 26, 'total_tokens': 126})

In [19]:
second_gemma_response.content

'Your name is Tareq. 😊  \n\nIs there anything else I can help you with, Tareq? \n'

In [20]:
first_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
first_prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [21]:
first_chain = first_prompt|gemma_model
first_chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [22]:
first_chain.invoke({"messages": [HumanMessage(content="Hey My name is Andrew.")]})

AIMessage(content="Hi Andrew, it's nice to meet you! How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 31, 'total_tokens': 55, 'completion_time': 0.043636364, 'prompt_time': 0.001486259, 'queue_time': 0.186252026, 'total_time': 0.045122623}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9e6539fd-528f-4d37-97e1-097c70d6669b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 24, 'total_tokens': 55})

In [23]:
with_message_history = RunnableWithMessageHistory(runnable=first_chain, get_session_history=get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000018E5FB91000>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [24]:
third_config = {"configurable": {"session_id": "chat3"}}
third_config

{'configurable': {'session_id': 'chat3'}}

In [25]:
first_prompt_response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is Andrew.")],
    config=third_config
)
first_prompt_response

AIMessage(content="Hello Andrew, it's nice to meet you!  What can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.001479638, 'queue_time': 0.18631228, 'total_time': 0.048752365}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--554155d0-f9b7-4117-bec6-6de27a9950da-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [26]:
second_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
second_prompt

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [27]:
second_chain = second_prompt|gemma_model
second_chain

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [28]:
second_prompt_response = second_chain.invoke({"messages": [HumanMessage(content="Hi, My name is Andrew.")], "language": "Arabic"})
second_prompt_response

AIMessage(content='مرحباً، أندرو! يسعدني اللقاء بك.  \n\nماذا يمكنني أن أفعل لك؟ 😊\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 34, 'total_tokens': 63, 'completion_time': 0.052727273, 'prompt_time': 0.001480478, 'queue_time': 0.186869709, 'total_time': 0.054207751}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7b8bf187-502e-476f-9f01-e4bd29903e0d-0', usage_metadata={'input_tokens': 34, 'output_tokens': 29, 'total_tokens': 63})

In [29]:
second_prompt_response.content

'مرحباً، أندرو! يسعدني اللقاء بك.  \n\nماذا يمكنني أن أفعل لك؟ 😊\n\n'

In [30]:
with_message_history = RunnableWithMessageHistory(
    runnable=second_chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  messages: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000018E5FB91000>, input_messages_key='messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [31]:
fourth_config = {"configurable": {"session_id": "chat4"}}
fourth_config

{'configurable': {'session_id': 'chat4'}}

In [32]:
third_prompt_response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am Andrew.")], "language": "Arabic"},
    config=fourth_config
)
third_prompt_response

AIMessage(content='مرحباً يا أندرو، أنا مساعد سعيد لتقديم المساعدة.  ما الذي يمكنني أن أفعل لك؟   \n(Hello Andrew, I am a happy assistant to help. What can I do for you?) \n\n\nLet me know how I can assist you! 😊\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 33, 'total_tokens': 96, 'completion_time': 0.114545455, 'prompt_time': 0.001485849, 'queue_time': 0.187135898, 'total_time': 0.116031304}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8646a87a-d5e6-4395-921d-613cc14683e4-0', usage_metadata={'input_tokens': 33, 'output_tokens': 63, 'total_tokens': 96})

In [33]:
third_prompt_response.content

'مرحباً يا أندرو، أنا مساعد سعيد لتقديم المساعدة.  ما الذي يمكنني أن أفعل لك؟   \n(Hello Andrew, I am a happy assistant to help. What can I do for you?) \n\n\nLet me know how I can assist you! 😊\n\n'

In [34]:
fourth_prompt_response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")], "language": "Arabic"},
    config=fourth_config
)
fourth_prompt_response

AIMessage(content='اسمك أندرو.  \n\n(Your name is Andrew.) \n\n\nIs there anything else I can help you with? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 109, 'total_tokens': 140, 'completion_time': 0.056363636, 'prompt_time': 0.002810108, 'queue_time': 0.187139986, 'total_time': 0.059173744}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ce77a8d8-bcb1-4650-a28a-dc476480e38f-0', usage_metadata={'input_tokens': 109, 'output_tokens': 31, 'total_tokens': 140})

In [35]:
fourth_prompt_response.content

'اسمك أندرو.  \n\n(Your name is Andrew.) \n\n\nIs there anything else I can help you with? 😊 \n'

In [36]:
trimmer = trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=gemma_model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
trimmer

RunnableLambda(...)

In [37]:
messages = [
    SystemMessage(content="You're a good assistant."),
    HumanMessage(content="Hi! I am Andrew."),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Nice!"),
    HumanMessage(content="What's the result of 2 + 2?"),
    AIMessage(content="The result is 4."),
    HumanMessage(content="Thanks!"),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]
messages

[SystemMessage(content="You're a good assistant.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am Andrew.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the result of 2 + 2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='The result is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [38]:
trimmer.invoke(messages)

h:\Data Scientists Projects\LLM-Powered Chatbot with Message History using Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="You're a good assistant.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am Andrew.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the result of 2 + 2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='The result is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [39]:
third_chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | second_prompt 
    | gemma_model
)
third_chain

RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.An

In [40]:
fifth_prompt_response = third_chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do I like?")],
        "language": "English"
    }
)
fifth_prompt_response

AIMessage(content='You like vanilla ice cream!  🍦\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 136, 'total_tokens': 147, 'completion_time': 0.02, 'prompt_time': 0.003673348, 'queue_time': 0.188902317, 'total_time': 0.023673348}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--922b13f7-c3c2-44e8-ba30-1beb4bb61356-0', usage_metadata={'input_tokens': 136, 'output_tokens': 11, 'total_tokens': 147})

In [41]:
fifth_prompt_response.content

'You like vanilla ice cream!  🍦\n'

In [42]:
sixth_prompt_response = third_chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did I ask for?")],
        "language": "English"
    }
)
sixth_prompt_response

AIMessage(content='You asked what the result of 2 + 2 is.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 137, 'total_tokens': 152, 'completion_time': 0.027272727, 'prompt_time': 0.003445978, 'queue_time': 0.187873696, 'total_time': 0.030718705}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e46f024-f36a-4743-8d64-c692f7c1443d-0', usage_metadata={'input_tokens': 137, 'output_tokens': 15, 'total_tokens': 152})

In [43]:
sixth_prompt_response.content

'You asked what the result of 2 + 2 is.'

In [44]:
with_message_history = RunnableWithMessageHistory(
    runnable=third_chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  messages: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000018E5FB91000>, input_messages_key='messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [45]:
fifth_config = {"configurable": {"session_id": "chat5"}}
fifth_config

{'configurable': {'session_id': 'chat5'}}

In [46]:
seventh_prompt_response = with_message_history.invoke(
    {"messages": messages + [HumanMessage(content="What ice cream do I like?")], "language": "English"},
    config=fifth_config
)
seventh_prompt_response

AIMessage(content='You like vanilla ice cream! 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 136, 'total_tokens': 145, 'completion_time': 0.016363636, 'prompt_time': 0.003750578, 'queue_time': 0.189500365, 'total_time': 0.020114214}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a0b24602-3824-4aa7-9533-a4e30705758a-0', usage_metadata={'input_tokens': 136, 'output_tokens': 9, 'total_tokens': 145})

In [47]:
seventh_prompt_response.content

'You like vanilla ice cream! 😊'

In [48]:
eighth_prompt_response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What math problem did I ask for?")], "language": "English"},
    config=fifth_config
)
eighth_prompt_response

AIMessage(content='You asked me what 2 + 2 equals!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 125, 'total_tokens': 138, 'completion_time': 0.023636364, 'prompt_time': 0.003293647, 'queue_time': 0.187093048, 'total_time': 0.026930011}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5596234c-f119-4184-9549-b1795a9190cf-0', usage_metadata={'input_tokens': 125, 'output_tokens': 13, 'total_tokens': 138})

In [49]:
eighth_prompt_response.content

'You asked me what 2 + 2 equals!'